In [33]:
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from gtts import gTTS
import tempfile
from pygame import mixer
import transform_stationname2

def say(text, filename=None):
    with tempfile.NamedTemporaryFile(delete=True) as temp:
        tts = gTTS(text, lang='zh-tw')
        if filename is None:
            filename = "{}.mp3".format(temp.name)
        tts.save(filename)
        mixer.init()
        mixer.music.load(filename)
        mixer.music.play()
        while mixer.music.get_busy() == True:
            continue
        mixer.quit()
        
def find_station(input_station):
    response = requests.get("https://tip.railway.gov.tw/tra-tip-web/tip/tip001/tip112/querybytime")
    soup = BeautifulSoup(response.text,"html.parser")
    station = soup.find('div', {'id': 'cityHot'})
    for name in station.find_all("button"):
        if input_station in name.get('title'):
            return name.get('title')
        
def find_train(startStation, endStation, rideDate=datetime.now().date(), current_time=datetime.now(), transfer="ONE", 
               trainTypeList="ALL", queryClassification="NORMAL"):
    # 語音辨識臺台轉換
    if startStation[0]=="台":
        startStation = startStation.replace("台", "臺")
    if endStation[0]=="台":
        endStation = endStation.replace("台", "臺")
    
    # find station name
    startStation = find_station(startStation)
    endStation = find_station(endStation)
    
    # data 格式轉換
    rideDate = str(rideDate).replace("-", "/")
    # 提取小時、分鐘
    current_hour = current_time.hour
    current_minute = current_time.minute
    # 如果分鐘超過 30，小時加 1
    if 0 < current_minute <= 30:
        current_minute = 30
    else:
        current_minute = 0
        current_hour += 1
    time_format = "00"
    startTime = str(current_hour).zfill(len(time_format))+":"+str(current_minute).zfill(len(time_format))
    endTime = str(current_hour+2).zfill(len(time_format))+":"+str(current_minute).zfill(len(time_format))
    
    request = requests.Session()
    response = request.get("https://tip.railway.gov.tw/tra-tip-web/tip/tip001/tip112/querybytime")
    soup = BeautifulSoup(response.text,"html.parser")
    _csrf = soup.find(attrs={"name":"_csrf"}).get('value')

    payload = {
        "_csrf": _csrf,
        "startStation": startStation,
        "endStation": endStation,
        "transfer": transfer,
        "rideDate": rideDate,
        "startOrEndTime": True,
        "startTime": startTime,
        "endTime": endTime,
        "trainTypeList": trainTypeList,
        "queryClassification": queryClassification,
        "query": "查詢"
    }
    # print(_csrf, startStation, endStation, rideDate, startTime, endTime)
    
    url = 'https://tip.railway.gov.tw/tra-tip-web/tip/tip001/tip112/querybytime'
    response = request.post(url, data=payload)
    soup = BeautifulSoup(response.text,"html.parser")
    detail_column = soup.find_all('tr', class_='trip-column')
    for count, data in enumerate(detail_column):
        if count>=5:
            break
        trainNumber = data.find('a').text+'('+data.find_all('span', class_='location')[0].text+'→'+data.find_all('span', class_='location')[1].text+')'
        startTime = data.find_all('td')[1].text
        endTime = data.find_all('td')[2].text
        duration = data.find_all('td')[3].text
        # upload to server and say
        url = "http://IP_address/main/Railway.php"
        data = {
            'StartStation': startStation,
            'EndStation': endStation,
            'TrainNumber': trainNumber,
            'rideDate': rideDate,
            'startTime': startTime,
            'endTime': endTime,
            'Duration': duration,
            'kind': '台鐵'
        } 
        requests.get(url, params=data)
        print('車次:', trainNumber, '出發時間:', startTime, '抵達時間:', endTime,'車程:', duration)
        say("車次"+trainNumber+"起始站"+startStation+"終點站"+endStation+"出發時間"+startTime+"抵達時間"+endTime+"車程需約"+duration)
if __name__ == "__main__":
    print('臺北到高雄')
    find_train('臺北', '高雄')
    print('新竹到台北')
    find_train('新竹', '台北')  

臺北到高雄
車次: 普悠瑪 131(南港→潮州) 出發時間: 10:00 抵達時間: 13:40 車程: 3 小時 40 分
車次: 自強(3000) 117(基隆→潮州) 出發時間: 10:20 抵達時間: 14:55 車程: 4 小時 35 分
車次: 自強(3000) 422(樹林→新左營) 出發時間: 10:40 抵達時間: 17:00 車程: 6 小時 20 分
車次: 自強 121(七堵→屏東) 出發時間: 11:00 抵達時間: 15:53 車程: 4 小時 53 分
車次: 自強(3000) 123(七堵→屏東) 出發時間: 11:58 抵達時間: 16:53 車程: 4 小時 55 分
新竹到台北
車次: 區間 2144(嘉義→基隆) 出發時間: 10:21 抵達時間: 11:58 車程: 1 小時 37 分
車次: 普悠瑪 162(臺東→南港) 出發時間: 10:41 抵達時間: 11:34 車程: 53 分
車次: 區間 1172(新竹→基隆) 出發時間: 10:44 抵達時間: 12:24 車程: 1 小時 40 分
車次: 自強 112(潮州→基隆) 出發時間: 10:52 抵達時間: 12:02 車程: 1 小時 10 分
車次: 區間 1178(苗栗→基隆) 出發時間: 11:04 抵達時間: 12:37 車程: 1 小時 33 分
